In [ ]:
import numpy as np, pandas as pd, os, re

FILE_PATH = '/content/Metrics Dataset - Sheet15.csv'
Y_COL = None
X_COLS = None
MIN_ROWS = 30

def coerce_numeric(df):
    out = df.copy()
    for c in out.columns:
        if out[c].dtype == object:
            s = out[c].astype(str).str.replace('\u00a0','', regex=False).str.replace(' ','', regex=False)
            s = s.str.replace(',', '.', regex=False)
            s = s.str.replace(r'[^0-9\.\-eE+]', '', regex=True)
            out[c] = pd.to_numeric(s, errors='coerce')
    return out

def pick_numeric(df, min_rows=MIN_ROWS):
    num_cols = [c for c in df.columns if pd.api.types.is_numeric_dtype(df[c])]
    if len(num_cols) < 2:
        return None, None, num_cols, pd.Series(dtype=int)
    counts = df[num_cols].notna().sum().sort_values(ascending=False)
    y = counts.index[0]
    x_pool = [c for c in counts.index if c != y and counts[c] >= min_rows]
    x = x_pool[:1]
    return y, x, num_cols, counts

if FILE_PATH.lower().endswith(('.xlsx', '.xls')):
    sheets = pd.read_excel(FILE_PATH, sheet_name=None, engine='openpyxl')
    candidates = []
    for name_sh, sh in sheets.items():
        df_num = coerce_numeric(sh)
        y, x, num_cols, counts = pick_numeric(df_num)
        if y and x and len(df_num[[y]+x].dropna()) >= MIN_ROWS:
            candidates.append((name_sh, y, x, len(df_num[[y]+x].dropna())))
    if not candidates:
        raise SystemExit('No sheet with >=2 numeric columns and enough rows')
    name_sh, Y_COL, X_COLS, nrows = sorted(candidates, key=lambda t: -t[3])[0]
    df = coerce_numeric(sheets[name_sh])
    print('Sheet:', name_sh, '| Y =', Y_COL, '| X =', X_COLS, '| N =', nrows)
else:
    df_raw = pd.read_csv(FILE_PATH, engine='python')
    df = coerce_numeric(df_raw)
    Y_COL, X_COLS, num_cols, counts = pick_numeric(df)
    if not Y_COL or not X_COLS:
        raise SystemExit('Not enough numeric columns. Provide another file or set Y_COL/X_COLS manually.')
    print('Numeric columns:', [c for c in df.columns if pd.api.types.is_numeric_dtype(df[c])])
    print('Y =', Y_COL, '| X =', X_COLS)

data = df[[Y_COL]+X_COLS].dropna()
print('Rows after cleaning:', len(data))

In [ ]:
import numpy as np
Y = data[Y_COL].to_numpy(float)
X = data[X_COLS].to_numpy(float)
N, k = len(Y), X.shape[1]
print('N =', N, '| k =', k)
print('Y head:', Y[:5])
print('X head:', X[:5].reshape(-1,k))

In [ ]:
import statsmodels.api as sm
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score

lin = LinearRegression().fit(X, Y)
Y_pred = lin.predict(X)
b0, b = lin.intercept_, lin.coef_
print('Model: Y = {:.6g}{}'.format(b0, ''.join([f' + ({b[i]:.6g})*{X_COLS[i]}' for i in range(k)])))
R2 = r2_score(Y, Y_pred)
Adj_R2 = 1 - (1 - R2) * (N - 1) / (N - k - 1)
den = np.where(Y==0, np.nan, Y)
MRE = np.abs((Y - Y_pred) / den)
MMRE = np.nanmean(MRE)
PRED25 = np.nanmean(MRE < 0.25) * 100
print('R2 =', round(R2,6), '| Adj_R2 =', round(Adj_R2,6), '| MMRE =', round(float(MMRE),6), '| PRED(0.25) =', round(float(PRED25),2), '%')
X_sm = sm.add_constant(X)
ols = sm.OLS(Y, X_sm).fit()
print(ols.summary())

In [ ]:
import matplotlib.pyplot as plt, seaborn as sns
from scipy import stats

resid = Y - Y_pred
print('Normaltest p =', stats.normaltest(resid).pvalue if N>=8 else np.nan)
print('Shapiro p =', stats.shapiro(resid).pvalue if N<=5000 else np.nan)

plt.figure(figsize=(6,4))
sns.histplot(resid, kde=True)
plt.tight_layout(); plt.show()

plt.figure(figsize=(6,4))
stats.probplot(resid, dist='norm', plot=plt)
plt.tight_layout(); plt.show()

In [ ]:
import matplotlib.pyplot as plt
if k == 1:
    order = np.argsort(X[:,0])
    plt.figure(figsize=(6,4))
    plt.scatter(X[:,0], Y, s=14)
    plt.plot(X[order,0], Y_pred[order], linewidth=2)
    plt.xlabel(X_COLS[0]); plt.ylabel(Y_COL)
    plt.tight_layout(); plt.show()
else:
    print('Skip line plot: k > 1')